In [1]:
!pip install transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:0000:01


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import time
import random
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer, models
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
import numpy as np
import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split

# Load your custom dataset
def load_custom_dataset(filename):
    data = []
    with open(filename, "r") as file:
        for line in file:
            question, response, answer, label = line.strip().split('\t')
            label = float(label) / 5.0  # Normalize the similarity score to [0, 1]
            data.append((response, answer, label))
    return data

In [4]:
# Split dataset into train, validation, and test
def split_dataset(data, valid_percentage, test_percentage):
    length = len(data)
    np.random.shuffle(data)
    train = data[:int(length * (1 - valid_percentage - test_percentage))]
    valid = data[int(length * (1 - valid_percentage - test_percentage)):int(length * (1 - test_percentage))]
    test = data[int(length * (1 - test_percentage)):]
    return train, valid, test

In [5]:
# Modify the dataset loading process
dataset_path = '/kaggle/input/dataset/expand.txt'
raw_dataset = load_custom_dataset(dataset_path)
train_data, val_data, test_data = split_dataset(raw_dataset, valid_percentage=0.1, test_percentage=0.1)

In [6]:
import pandas as pd
import os

# Save datasets to CSV files
def save_splits_to_csv(train_data, val_data, test_data, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    train_df = pd.DataFrame(train_data, columns=["response", "answer", "label"])
    val_df = pd.DataFrame(val_data, columns=["response", "answer", "label"])
    test_df = pd.DataFrame(test_data, columns=["response", "answer", "label"])

    train_df.to_csv(os.path.join(output_dir, "train_data.csv"), index=False)
    val_df.to_csv(os.path.join(output_dir, "val_data.csv"), index=False)
    test_df.to_csv(os.path.join(output_dir, "test_data.csv"), index=False)

    print(f"Data saved to {output_dir} successfully.")

# Load datasets from CSV files
def load_splits_from_csv(output_dir):
    train_df = pd.read_csv(os.path.join(output_dir, "train_data.csv"))
    val_df = pd.read_csv(os.path.join(output_dir, "val_data.csv"))
    test_df = pd.read_csv(os.path.join(output_dir, "test_data.csv"))

    # Convert dataframes back to lists of tuples
    train_data = list(train_df.itertuples(index=False, name=None))
    val_data = list(val_df.itertuples(index=False, name=None))
    test_data = list(test_df.itertuples(index=False, name=None))

    print(f"Data loaded from {output_dir} successfully.")
    return train_data, val_data, test_data

# Save splits to CSV
output_directory = "output_splits"
save_splits_to_csv(train_data, val_data, test_data, output_directory)

Data saved to output_splits successfully.


In [8]:
# Load splits from CSV
train_data, val_data, test_data = load_splits_from_csv(output_directory)

# Check the first few rows to ensure data integrity
print("Train Data (First 5 Rows):", train_data[:5])
print("Validation Data (First 5 Rows):", val_data[:5])
print("Test Data (First 5 Rows):", test_data[:5])

Data loaded from output_splits successfully.
Train Data (First 5 Rows): [('To simulate portions of the desired final product with a quick and easy program that does a small specific job It is a way to help see what the problem is and how you may solve it in the final project', 'A program that simulates the behavior of portions of the desired software product', 1.0), ('Both involve a controlled repetition structures and they both have a termination test Also both of them can loop forever', 'anything you can do iterativly you can do recursively', 0.6), ('Taking one array element at a time from left to right it identifies the minimum from the remaining elements and swaps it with the current element', 'the sort finds the largest item and places it at the end then then next largest is found and placed next to last continue until there is only one number left this is the beginning of the sorted array', 0.9), ('Find the node then replace it with the leftmost node from its right subtree - LRB 

In [10]:
# Instantiate the BERT tokenizer
# You can use larger variants of the model, here we're using the base model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [12]:
# Correct the CustomDataset __getitem__ method
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.first_sentences = [pair[0] for pair in data]
        self.second_sentences = [pair[1] for pair in data]
        self.labels = [pair[2] for pair in data]

    def __len__(self):
        return len(self.first_sentences)

    def __getitem__(self, idx):
        texts = tokenizer(
            self.first_sentences[idx],
            self.second_sentences[idx],
            padding="max_length",
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return {
            'input_ids': texts['input_ids'].squeeze(0),
            'attention_mask': texts['attention_mask'].squeeze(0),
        }, label

In [13]:
# Instantiate dataset objects
train_ds = CustomDataset(train_data)
val_ds = CustomDataset(val_data)

In [14]:
# DataLoaders
batch_size = 8
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_ds, batch_size=batch_size)

In [15]:
class EnhancedBertModel(nn.Module):
    def __init__(self):
        super(EnhancedBertModel, self).__init__()
        # Load the pre-trained BERT model
        self.bert = models.Transformer('bert-base-uncased', max_seq_length=128)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        
        # Freeze BERT layers initially
        for param in self.bert.parameters():
            param.requires_grad = False

        # Additional trainable BiLSTM layer
        self.bi_lstm = nn.LSTM(
            input_size=self.bert.get_word_embedding_dimension(),
            hidden_size=64,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        
        # Fully connected layers
        self.fc_dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(256, 1)  # Input dimension updated to match concatenated pooling outputs

    def forward(self, input_data):
        # Get embeddings from BERT
        bert_output = self.bert(input_data)
        
        # Extract token embeddings (last hidden states)
        sequence_output = bert_output['token_embeddings']  # Replace with 'last_hidden_state' if using HuggingFace models directly

        # BiLSTM layer
        lstm_output, _ = self.bi_lstm(sequence_output)

        # Pooling layers
        avg_pool = torch.mean(lstm_output, dim=1)
        max_pool, _ = torch.max(lstm_output, dim=1)

        # Concatenate pooling results
        pooled_output = torch.cat((avg_pool, max_pool), dim=1)

        # Fully connected layer with dropout
        output = self.fc_dropout(pooled_output)
        output = self.fc(output)

        return output.squeeze()


In [16]:
# Check for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [19]:
# Instantiate and move the model to device
model = EnhancedBertModel()
model.to(device)

EnhancedBertModel(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (bi_lstm): LSTM(768, 64, batch_first=True, bidirectional=True)
  (fc_dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [20]:
# Loss function, optimizer, and scheduler
criterion = nn.MSELoss()
epochs = 8
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [21]:
# Training Loop
def train_model():
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(epochs):
        print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
        print("Training...")

        t0 = time.time()
        total_train_loss = 0
        model.train()

        for batch in tqdm(train_dataloader):
            train_data, train_labels = batch
            train_data['input_ids'] = train_data['input_ids'].to(device)
            train_data['attention_mask'] = train_data['attention_mask'].to(device)
            train_labels = train_labels.to(device)

            optimizer.zero_grad()
            outputs = model({
                'input_ids': train_data['input_ids'],
                'attention_mask': train_data['attention_mask']
            })
            loss = criterion(outputs, train_labels)
            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        avg_train_loss = total_train_loss / len(train_dataloader)
        training_time = format_time(time.time() - t0)

        print(f"  Average training loss: {avg_train_loss:.5f}")
        print(f"  Training epoch took: {training_time}")

        # Validation
        print("Running Validation...")
        t0 = time.time()

        model.eval()
        total_val_loss = 0

        for batch in tqdm(validation_dataloader):
            val_data, val_labels = batch
            val_data['input_ids'] = val_data['input_ids'].to(device)
            val_data['attention_mask'] = val_data['attention_mask'].to(device)
            val_labels = val_labels.to(device)

            with torch.no_grad():
                outputs = model({
                    'input_ids': val_data['input_ids'],
                    'attention_mask': val_data['attention_mask']
                })
                loss = criterion(outputs, val_labels)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(validation_dataloader)
        validation_time = format_time(time.time() - t0)

        print(f"  Validation Loss: {avg_val_loss:.5f}")
        print(f"  Validation took: {validation_time}")

        training_stats.append({
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validation Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        })

    print("Training complete!")
    print(f"Total training took {format_time(time.time() - total_t0)}")
    return model, training_stats


In [22]:
# Train the model
model, training_stats = train_model()


======== Epoch 1 / 8 ========
Training...


 30%|███       | 111/365 [00:08<00:14, 17.12it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 76%|███████▌  | 277/365 [00:18<00:05, 16.91it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed

  Average training loss: 0.18951
  Training epoch took: 0:00:23
Running Validation...


100%|██████████| 46/46 [00:02<00:00, 17.88it/s]


  Validation Loss: 0.06160
  Validation took: 0:00:03

======== Epoch 2 / 8 ========
Training...


...

  Validation Loss: 0.04852
  Validation took: 0:00:03

======== Epoch 8 / 8 ========
Training...


 43%|████▎     | 158/365 [00:10<00:13, 15.08it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 52%|█████▏    | 190/365 [00:12<00:11, 15.09it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed

  Average training loss: 0.05858
  Training epoch took: 0:00:24
Running Validation...


100%|██████████| 46/46 [00:02<00:00, 15.81it/s]

  Validation Loss: 0.04871
  Validation took: 0:00:03
Training complete!
Total training took 0:03:30


In [23]:
# Fine-tune BERT layers
for param in model.bert.parameters():
    param.requires_grad = True

In [24]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [25]:
# Retrain the entire model
model, training_stats = train_model()


======== Epoch 1 / 8 ========
Training...


100%|██████████| 365/365 [01:15<00:00,  4.82it/s]


  Average training loss: 0.05113
  Training epoch took: 0:01:16
Running Validation...


100%|██████████| 46/46 [00:02<00:00, 15.68it/s]


  Validation Loss: 0.04652
  Validation took: 0:00:03

======== Epoch 2 / 8 ========
Training...


...

  Validation Loss: 0.01912
  Validation took: 0:00:03

======== Epoch 8 / 8 ========
Training...


100%|██████████| 365/365 [01:15<00:00,  4.82it/s]


  Average training loss: 0.00892
  Training epoch took: 0:01:16
Running Validation...


100%|██████████| 46/46 [00:02<00:00, 15.53it/s]

  Validation Loss: 0.01886
  Validation took: 0:00:03
Training complete!
Total training took 0:10:29


In [26]:
# Define custom evaluation functions
def mean_squared_error(y_true, y_pred):
    squared_errors = [(true - pred) ** 2 for true, pred in zip(y_true, y_pred)]
    return sum(squared_errors) / len(squared_errors)

def mean_absolute_error(y_true, y_pred):
    absolute_errors = [abs(true - pred) for true, pred in zip(y_true, y_pred)]
    return sum(absolute_errors) / len(absolute_errors)

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return mse ** 0.5

def pearsonr(x, y):
    mean_x = sum(x) / len(x)
    mean_y = sum(y) / len(y)
    numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    denominator = ((sum((xi - mean_x) ** 2 for xi in x) * sum((yi - mean_y) ** 2 for yi in y)) ** 0.5)
    return numerator / denominator if denominator != 0 else 0.0

In [27]:
# Evaluate Model with custom functions
def evaluate_model(model, dataloader):
    model.eval()
    true_labels = []
    predicted_scores = []

    with torch.no_grad():
        for batch in dataloader:
            data, labels = batch
            data['input_ids'] = data['input_ids'].to(device)
            data['attention_mask'] = data['attention_mask'].to(device)
            predictions = model({
                'input_ids': data['input_ids'],
                'attention_mask': data['attention_mask']
            })

            true_labels.extend(labels.cpu().numpy())
            predicted_scores.extend(predictions.cpu().numpy())

    mse = mean_squared_error(true_labels, predicted_scores)
    mae = mean_absolute_error(true_labels, predicted_scores)
    rmse = root_mean_squared_error(true_labels, predicted_scores)
    pearson_corr = pearsonr(true_labels, predicted_scores)

    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Pearson Correlation: {pearson_corr:.4f}")

    return mse, mae, rmse, pearson_corr

# Evaluate on validation set
evaluate_model(model, validation_dataloader)

# Optional: Evaluate on the test set if available
test_ds = CustomDataset(test_data)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)
print("this is the evaluation on the test set:")
evaluate_model(model, test_dataloader)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Mean Squared Error (MSE): 0.0186
Mean Absolute Error (MAE): 0.0923
Root Mean Squared Error (RMSE): 0.1362
Pearson Correlation: 0.8355
this is the evaluation on the test set:


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Mean Squared Error (MSE): 0.0179
Mean Absolute Error (MAE): 0.0948
Root Mean Squared Error (RMSE): 0.1337
Pearson Correlation: 0.8242


(0.017877860220981413,
 0.09482546901355868,
 0.13370811576333508,
 0.824246506790285)

In [28]:
# Save the trained model to a .pt file
model_save_path = "/kaggle/working/bert_similarity_model.pt"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /kaggle/working/bert_similarity_model.pt


In [29]:
# Reinitialize the model architecture
model = EnhancedBertModel()
model.to(device)

# Load the model state dictionary
model.load_state_dict(torch.load(model_save_path))
print("Model loaded successfully!")

# Set the model to evaluation mode if testing
model.eval()


/tmp/ipykernel_30/3771432145.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


Model loaded successfully!


EnhancedBertModel(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (bi_lstm): LSTM(768, 64, batch_first=True, bidirectional=True)
  (fc_dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [30]:
def evaluate_and_save_results(model, dataloader, csv_filename, original_data):
    model.eval()
    results = []  # To store all prediction results
    index = 0  # Track the original dataset index

    with torch.no_grad():
        for batch in tqdm(dataloader):
            data, labels = batch
            data['input_ids'] = data['input_ids'].to(device)
            data['attention_mask'] = data['attention_mask'].to(device)
            predictions = model({
                'input_ids': data['input_ids'],
                'attention_mask': data['attention_mask']
            }).cpu().numpy()

            batch_size = len(labels)
            for i in range(batch_size):
                # Get original sentences (response, answer)
                response, answer, true_label = original_data[index]
                index += 1  # Move to the next pair

                results.append({
                    'Response': response,
                    'Answer': answer,
                    'True Label': true_label,
                    'Predicted Score': predictions[i]
                })

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")
    
    return df

# Save validation results
validation_results_csv = "/kaggle/working/validation_results.csv"
df_validation = evaluate_and_save_results(model, validation_dataloader, validation_results_csv, val_data)

# Load test data for testing
test_ds = CustomDataset(test_data)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

# Save test results
test_results_csv = "/kaggle/working/test_results.csv"
df_test = evaluate_and_save_results(model, test_dataloader, test_results_csv, test_data)


100%|██████████| 46/46 [00:02<00:00, 16.10it/s]


Results saved to /kaggle/working/validation_results.csv


100%|██████████| 46/46 [00:02<00:00, 16.15it/s]

Results saved to /kaggle/working/test_results.csv
